---
# Vacation_Search
---

## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chuy,UY,-33.6971,-53.4616,70.11,81,16,12.12,few clouds
1,1,Lebu,CL,-37.6167,-73.6500,60.03,71,27,12.59,scattered clouds
2,2,Cape Town,ZA,-33.9258,18.4232,66.00,88,0,17.27,clear sky
3,3,Kavieng,PG,-2.5744,150.7967,79.38,81,96,6.46,moderate rain
4,4,Hargeysa,SO,9.5600,44.0650,66.81,76,23,7.25,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[ ( city_data_df['Max Temp'] >= min_temp ) &
                                        ( city_data_df['Max Temp'] <= max_temp ) ]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                116
City                   116
Country                115
Lat                    116
Lng                    116
Max Temp               116
Humidity               116
Cloudiness             116
Wind Speed             116
Current Description    116
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                115
City                   115
Country                115
Lat                    115
Lng                    115
Max Temp               115
Humidity               115
Cloudiness             115
Wind Speed             115
Current Description    115
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Lebu,CL,60.03,scattered clouds,-37.6167,-73.6500,
2,Cape Town,ZA,66.00,clear sky,-33.9258,18.4232,
4,Hargeysa,SO,66.81,few clouds,9.5600,44.0650,
8,Bethel,US,60.80,clear sky,41.3712,-73.4140,
13,Albany,US,60.01,broken clouds,42.6001,-73.9662,
14,Castro,BR,63.93,scattered clouds,-24.7911,-50.0119,
27,Pacific Grove,US,60.01,clear sky,36.6177,-121.9166,
34,Port Elizabeth,ZA,66.20,overcast clouds,-33.9180,25.5701,
36,Seddon,AU,62.01,overcast clouds,-37.8068,144.8916,
39,Hermanus,ZA,64.99,light rain,-34.4187,19.2345,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): 
        print("Hotel not found... skipping.")

print("COMPLETED")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
COMPLETED


In [9]:
# 7. Drop the rows where there is no Hotel Name.
# 7a. Replace blanks with NaN
hotel_df["Hotel Name"].replace( "" , np.nan ,inplace = True )
# 7b. Drop NaN rows
clean_hotel_df = hotel_df.dropna( axis = 0 , subset=['Hotel Name'] )

In [10]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Lebu,CL,60.03,scattered clouds,-37.6167,-73.6500,Hostal Las Lilas
2,Cape Town,ZA,66.00,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
4,Hargeysa,SO,66.81,few clouds,9.5600,44.0650,City Center Hotel
8,Bethel,US,60.80,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
14,Castro,BR,63.93,scattered clouds,-24.7911,-50.0119,CHACARA BAILLY
...,...,...,...,...,...,...,...
661,Chokwe,MZ,66.45,clear sky,-24.5333,32.9833,Complexo Sonho Real
670,Ballina,AU,68.00,broken clouds,-28.8667,153.5667,Ramada Hotel & Suites by Wyndham Ballina Byron
671,Durban,ZA,68.00,clear sky,-29.8579,31.0292,Hilton Durban
689,Patea,NZ,64.99,overcast clouds,-39.7583,174.4833,Patea Rest N Fresh


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, 
#    the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))